# Производительность модели семейства столбцов на примере работы с MonetDB

<p> https://en.wikipedia.org/wiki/MonetDB
<p> http://pymonetdb.readthedocs.io/en/1.1.1/

|  reg_id | reg_name |   year  | group_id | group_name | categry | gender |  value  |
|:-------:|:--------:|:-------:|:--------:|:----------:|:-------:|:------:|:-------:|
| integer |  string  | integer |  integer |   string   |  string | string | integer |

In [1]:
! mserver5 --version | head -n1

MonetDB 5 server v11.29.3 "Mar2018" (64-bit, 128-bit integers)


In [2]:
import sys
import time

import pymonetdb

sys.path.append('../config/')
from config import config

In [3]:
params = config(section='monetdb')
conn = pymonetdb.connect(
    database=params["database"],
    hostname=params["hostname"],
    username=params["username"],
    password=params["password"])
cur = conn.cursor()
cur.arraysize = 100

In [4]:
commands = (
    """
    SELECT *
    FROM "crimestatsocial";""",

    """
    SELECT *
    FROM "crimestatsocial"
    WHERE "year"=2016;""",

     """
    SELECT *, COUNT("group_id")
    FROM "crimestatsocial"
    WHERE "year"=2016
    GROUP BY "group_id";""",

    """
    SELECT "year", COUNT("year")
    FROM "crimestatsocial"
    GROUP BY "year";""",

    """
    SELECT "group_id", COUNT(DISTINCT "group_id")
    FROM "crimestatsocial"
    GROUP BY "group_id";""",
    
    """
    INSERT INTO "crimestatsocial"
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);""",
    
    """
    UPDATE "crimestatsocial"
    SET "year" = 2017, "value" = 1
    WHERE "reg_id" = %s;""",
    
    """
    DELETE FROM "crimestatsocial"
    WHERE "reg_id" = %s;""",
)

## 1. Собираются исходные кортежи

In [5]:
cur.execute(commands[0])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

27384
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины', 13573)


In [6]:
%timeit cur.execute(commands[0])

18 ms ± 418 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 2. Собираются исходные кортежи за 2016 год

In [7]:
cur.execute(commands[1])
fetchall = cur.fetchall()
print(len(fetchall))
print(fetchall[0])

3948
(1100, 'Российская Федерация', 2016, 14002, 'по возрасту', '14-15', 'Мужчины', 13573)


In [8]:
%timeit cur.execute(commands[1])

20.8 ms ± 1.63 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 3. Подсчет исходных кортежей за 2016 год по группам

In [9]:
cur.execute(commands[2])
cur.fetchall()

[(14002, 1128), (14006, 1692), (14005, 752), (14007, 376)]

In [10]:
%timeit cur.execute(commands[2])

6.17 ms ± 580 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 4. Подсчет исходных кортежей по годам

In [11]:
cur.execute(commands[3])
cur.fetchall()

[(2016, 3948),
 (2015, 3990),
 (2014, 3990),
 (2013, 3864),
 (2012, 3864),
 (2011, 3864),
 (2010, 3864)]

In [12]:
%timeit cur.execute(commands[3])

7.07 ms ± 666 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 5. Подсчет уникальных групп

In [13]:
cur.execute(commands[4])

4

In [14]:
%timeit cur.execute(commands[4])

8.22 ms ± 697 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 6. Вставка элементов столбцов

In [15]:
item = (int(1e7) - 1, 'text', 0, 0, 'text', 'text', 'text', 0)
cur.execute(commands[5], item)
conn.commit()

-1

In [16]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[5], (i, 'text', 0, 0, 'text', 'text', 'text', 0, ))
    i += 1

In [17]:
%timeit -n 1000 -r 5 fun()

1.98 ms ± 203 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [18]:
conn.commit()

-1

## 7. Обновление элементов столбцов

In [19]:
cur.execute(commands[6], (int(1e7) - 1, ))
conn.commit()

-1

In [20]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[6], (i, ))
    i += 1

In [21]:
%timeit -n 1000 -r 5 fun()

2.96 ms ± 104 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [22]:
conn.commit()

-1

## 8. Удаление элементов столбцов

In [23]:
cur.execute(commands[7], (int(1e7) - 1, ))
conn.commit()

-1

In [24]:
i = int(1e7)
def fun():
    global i
    cur.execute(commands[7], (i, ))
    i += 1

In [25]:
%timeit -n 1000 -r 5 fun()

2.1 ms ± 145 µs per loop (mean ± std. dev. of 5 runs, 1000 loops each)


In [26]:
conn.commit()

-1

In [27]:
cur.close()
conn.close()